In [13]:
import requests


# Page 1 of 7 (63 results, 10 per page)
tapei = 'https://www.ncei.noaa.gov/access/services/search/v1/data?bbox=25.404,120.010,21.719,122.537&startDate=2017-01-01T00:00:00&endDate=2021-12-31T23:59:59&dataset=global-hourly&limit=10000'

# Page 1 of 3 (30 results, 10 per page)
delhi = 'https://www.ncei.noaa.gov/access/services/search/v1/data?bbox=29.124,76.516,27.813,78.099&startDate=2017-01-01T00:00:00&endDate=2021-12-31T23:59:59&dataset=global-hourly&limit=10000'

# Page 1 of 17 (168 results, 10 per page)
los_angeles = 'https://www.ncei.noaa.gov/access/services/search/v1/data?bbox=34.581,-119.202,33.281,-116.295&startDate=2017-01-01T00:00:00&endDate=2021-12-31T23:59:59&dataset=global-hourly&limit=10000'
base_path = "https://www.ncei.noaa.gov"

locations = [(tapei, "tp"), (delhi, "dh"), (los_angeles, "la")]
for location, folder in locations:
    r = requests.get(location).json()
    print(f"Prcessing {folder}")
    for result in r['results']:
        file_path = base_path+result['filePath']
        file = requests.get(file_path, allow_redirects=True).content
        file_name = f"{folder}/"+"-".join(result['filePath'].split("/")[-2:])
        f = open(file_name, "w")
        f.write(str(file, 'utf-8'))
        f.close()


Prcessing tp
Prcessing dh
Prcessing la


In [6]:
from pathlib import Path
import pandas as pd

p = Path("tp").glob("*.csv")
files = [x for x in p if x.is_file()]
dfs = [pd.read_csv(i) for i in files]

df = pd.concat(dfs)
df.to_csv("tp_2017_2021.csv", index=False)

match a weather station to the nearest pm station

In [2]:
import pandas as pd
metadata = pd.read_csv("/mnt/d/airdata/metadata_updated.csv")

files = ["/mnt/d/airdata/noaa_redownload/dh_2017_2021.csv", "/mnt/d/airdata/noaa_redownload/tp_2017_2021.csv", "/mnt/d/airdata/noaa_redownload/la_2017_2021.csv"]
dfs = [pd.read_csv(i).drop_duplicates('STATION') for i in files]

weather_df = pd.concat(dfs)

/tmp/ipykernel_2356/4196525566.py:4: DtypeWarning: Columns (30,31,32,35,37,38,39,40,41,42,43) have mixed types.Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(i).drop_duplicates('STATION') for i in files]
/tmp/ipykernel_2356/4196525566.py:4: DtypeWarning: Columns (16,20,25,26,28,29,30,31,32,33,34,35) have mixed types.Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(i).drop_duplicates('STATION') for i in files]
/tmp/ipykernel_2356/4196525566.py:4: DtypeWarning: Columns (0,2,8,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132) have mixed types.Specify dtype option on import or set low_memory=False.
  dfs

In [3]:
def find_nearest(row, station=None):
    long = row['LONGITUDE']
    lat = row['LATITUDE']
    metadata['dist'] = ((abs(metadata['Latitudes']) - abs(lat))**2 + (abs(metadata['Longitudes']) -abs(long)) **2)**0.5
    smallest_row = metadata[metadata['dist'] == metadata['dist'].min()]
    if station:
        return smallest_row['dist'].values[0]
    return smallest_row['grid_id'].values[0]

weather_df['grid_id'] = weather_df.apply(lambda x: find_nearest(x), axis=1)
weather_df['station_distance'] = weather_df.apply(lambda x: find_nearest(x, station=True), axis=1)

In [4]:
weather_df = weather_df[['STATION', 'grid_id', 'station_distance']]

In [16]:
metadata_with_station = weather_df.sort_values(['grid_id', 'station_distance']).drop_duplicates('grid_id')

In [21]:
metadata.merge(metadata_with_station, on='grid_id').to_csv('metadata_with_station.csv', index=False)

keep only columns that have the appropriate id

In [1]:
import pandas as pd
metadata = pd.read_csv("/mnt/d/airdata/metadata_updated.csv")
metadata[metadata['location'] == 'Los Angeles (SoCAB)'].to_csv('la_metadata.csv', index=False)

In [2]:
la_metadata = metadata[metadata['location'] == 'Los Angeles (SoCAB)']

In [3]:
la_df = pd.read_csv('la_2017_2021.csv').drop_duplicates('STATION')

/home/ding/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (0,2,8,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
def find_nearest(row, metadata, station=None):
    long = row['LONGITUDE']
    lat = row['LATITUDE']
    metadata['dist'] = ((abs(metadata['Latitudes']) - abs(lat))**2 + (abs(metadata['Longitudes']) -abs(long)) **2)**0.5
    smallest_row = metadata[metadata['dist'] == metadata['dist'].min()]
    if station:
        return smallest_row['dist'].values[0]
    return smallest_row['grid_id'].values[0]

la_df['grid_id'] = la_df.apply(lambda x: find_nearest(x, la_metadata), axis=1)
la_df['station_distance'] = la_df.apply(lambda x: find_nearest(x,la_metadata, station=True), axis=1)


In [9]:
la_metadata = la_df[['STATION', 'grid_id', 'station_distance']].sort_values(['grid_id', 'station_distance']).drop_duplicates('grid_id')
la_metadata.to_csv('la_metadata.csv', index=False)

In [13]:
la_metadata.reset_index(drop=True, inplace=True)

In [16]:
la_df = pd.read_csv('la_2017_2021.csv')

/home/ding/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (0,2,8,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [18]:
la_df_reduced_stations = la_df[la_df['STATION'].isin(la_metadata['STATION'])]

In [19]:
la_df_reduced_stations.to_csv('la_2017_2021_unique_station.csv', index=False)